In [2]:
from Re0_neural_network import activations, initializers, layers, losses, models, optimizers, metrics

import numpy as np
import matplotlib.pyplot as plt
import time

# load dataset
mnist = np.load('datasets/mnist.npz')
train_images = mnist['train_images']
train_labels = mnist['train_labels']
test_images = mnist['test_images']
test_labels = mnist['test_labels']

# add 1 dim to the last axis of the image
train_images = train_images.reshape(*(train_images.shape), 1).astype(float)
test_images = test_images.reshape(*(test_images.shape), 1).astype(float)

# normalisation
train_images /= 255
test_images /= 255

# one-hot encode
targets = train_labels.reshape(-1)
train_labels = np.eye(10)[targets].reshape(60000, 10, 1)

targets = test_labels.reshape(-1)
test_labels = np.eye(10)[targets].reshape(10000, 10, 1)



(60000,)

IndexError: arrays used as indices must be of integer (or boolean) type

(60000, 10, 1)

In [3]:
epochs = range(1, 6)

train_losses = []
train_metrics = []
val_losses = []
val_metrics = []

model =models.Sequential()
model.append(layers.Flatten())
model.append(layers.Dense(784, 64, activations.relu, initializers.He))
model.append(layers.Dense(64, 10, activations.softmax, initializers.He))
model.build()

model.config(optimizer=optimizers.SGD, 
             loss=losses.Crossentropy,
             lr=0.001,
             batch_size=20,
             metric=metrics.categorical_accuracy)
for epoch in epochs:
    begin_time = time.time()
    train_loss, train_metric, val_loss, val_metric = model.fit(train_images, train_labels, 0.001)

    train_losses.append(train_loss)
    train_metrics.append(train_metric)
    val_losses.append(val_loss)
    val_metrics.append(val_metric)
    print(
        "epoch: {} train_loss: {:.3f} train_accuracy: {:.2%} val_loss: {:.3f} val_accuracy: {:.2%} time_per_epoch: {:.1f}"
        .format(epoch, train_loss, train_metric, val_loss, val_metric,
                time.time() - begin_time))

              Model summary                
layer 1 Flatten    parameters: 0         
layer 2 Dense      parameters: 50240     
layer 3 Dense      parameters: 650       
total parameters: 50890
Batch size: 20
Loss: Crossentropy
metric: categorical_accuracy
learning rate: 0.001
optimizer: SGD
epoch: 1 train_loss: 1.598 train_accuracy: 58.83% val_loss: 1.598 val_accuracy: 59.20% time_per_epoch: 57.3
epoch: 2 train_loss: 0.831 train_accuracy: 81.38% val_loss: 0.828 val_accuracy: 81.55% time_per_epoch: 48.9
epoch: 3 train_loss: 0.608 train_accuracy: 85.15% val_loss: 0.600 val_accuracy: 85.53% time_per_epoch: 53.7
epoch: 4 train_loss: 0.511 train_accuracy: 87.08% val_loss: 0.512 val_accuracy: 87.07% time_per_epoch: 49.7
epoch: 5 train_loss: 0.456 train_accuracy: 88.26% val_loss: 0.462 val_accuracy: 87.44% time_per_epoch: 49.3


In [4]:
epochs = range(1, 6)

train_losses = []
train_metrics = []
val_losses = []
val_metrics = []
model =models.Sequential()
model.append(layers.Flatten())
model.append(layers.Dense(784, 64, activations.relu, initializers.He))
model.append(layers.Dense(64, 10, activations.softmax, initializers.He))
model.build()

model.config(optimizer=optimizers.PBCSCD, 
             loss=losses.Crossentropy,
             lr=0.001,
             batch_size=20,
             metric=metrics.categorical_accuracy)
for epoch in epochs:
    begin_time = time.time()
    train_loss, train_metric, val_loss, val_metric = model.fit(train_images, train_labels, 0.001)

    train_losses.append(train_loss)
    train_metrics.append(train_metric)
    val_losses.append(val_loss)
    val_metrics.append(val_metric)
    print(
        "epoch: {} train_loss: {:.3f} train_accuracy: {:.2%} val_loss: {:.3f} val_accuracy: {:.2%} time_per_epoch: {:.1f}"
        .format(epoch, train_loss, train_metric, val_loss, val_metric,
                time.time() - begin_time))

              Model summary                
layer 1 Flatten    parameters: 0         
layer 2 Dense      parameters: 50240     
layer 3 Dense      parameters: 650       
total parameters: 50890
Batch size: 20
Loss: Crossentropy
metric: categorical_accuracy
learning rate: 0.001
optimizer: PBCSCD
epoch: 1 train_loss: 1.560 train_accuracy: 58.62% val_loss: 1.559 val_accuracy: 58.43% time_per_epoch: 52.1
epoch: 2 train_loss: 0.794 train_accuracy: 82.84% val_loss: 0.783 val_accuracy: 83.58% time_per_epoch: 56.1
epoch: 3 train_loss: 0.595 train_accuracy: 85.59% val_loss: 0.581 val_accuracy: 86.34% time_per_epoch: 57.1
epoch: 4 train_loss: 0.501 train_accuracy: 86.96% val_loss: 0.497 val_accuracy: 87.21% time_per_epoch: 55.4
epoch: 5 train_loss: 0.450 train_accuracy: 88.18% val_loss: 0.439 val_accuracy: 88.59% time_per_epoch: 52.7


In [ ]:
3/2

In [ ]:
[x,for x in range(3)]

In [ ]:
l = [x for x in range( 11)]

In [ ]:
l.remove(3)
l

In [ ]:
l.remove(7)
l

In [ ]:
class PBCSCD(Optimizer):
    """
    "pseudo" Block-Cyclic Stochastic Coordinate 
    
    """
    
    
    def step(self, x, y):
        k = self.model.batch_size//(self.layer_numbers+1)##number samples used to calculate gradient
        L_layer = [x for x in range(self.layer_numbers)] ##liste of number of layers
        
        for i in range (self.layer_numbers):
            layer_i = L_layer[np.random.randint(len(L_layer))]##randomly choose a layer
            x_i = x[i*k:(i+1)*k]## samples used to calculate gradient
            y_i = y[i*k:(i+1)*k]
           
            super().step(x_i, y_i)
            
            self.P[layer_i]['w'] -= self.lr * self.G[layer_i]['w']##update parameters of the randomly chosen layer
            self.P[layer_i]['b'] -= self.lr * self.G[layer_i]['b']
            
            L_layer.remove(layer_i) ## the layer whos parameters that have been updated will be removed from the list so that all parameters will be updated in one epoch
        return (self.loss, self.metric)